# Chess Gameplay

First import our `chess_gameplay` module.

In [33]:
import chess_gameplay as chg

## Playing a game

Let's take a look at how games will be played in the tournament. Before running this cell, open the `demo.png` file alongside so you can watch the action!

In [ ]:
from pathlib import Path
import re

# Define the path
path = Path("/shared/artifacts/3daf7d2b-d49a-412f-9428-94e28dd9e1ce/checkpoints")
copy_winner = '/shared/Winning/Model_4'

# Function to extract numerical part from "CHK" directories
def extract_number(dir_name):
    match = re.search(r'CHK(\d+)', dir_name)
    return int(match.group(1)) if match else float('inf')

# List and sort directories
chk_dirs = sorted(
    [d.name for d in path.iterdir() if d.is_dir() and d.name.startswith("CHK")],
    key=extract_number
)

print("Sorted directories starting with 'CHK':")
print(chk_dirs)

#checkpoints_location = {"check1": "/shared/yogesh/base_model_run_1/CHK0/checkpoint.pt" , 
#                        "check2": "/shared/yogesh/base_model_run_1/CHK1/checkpoint.pt" ,
#                        "check3": "/shared/yogesh/base_model_run_1/CHK2/checkpoint.pt"}
checkpoints_location = {}

for dir in chk_dirs:
    checkpoints_location[dir] = str(path) + "/" + dir + "/checkpoint.pt"

print(checkpoints_location)


Sorted directories starting with 'CHK':
['CHK0', 'CHK1', 'CHK2']
{'CHK0': '/shared/yogesh/base_model_run_1/CHK0/checkpoint.pt', 'CHK1': '/shared/yogesh/base_model_run_1/CHK1/checkpoint.pt', 'CHK2': '/shared/yogesh/base_model_run_1/CHK2/checkpoint.pt'}


## Agents using models

Until you have trained a model and saved a checkpoint, you will not be able to run the following cell, but you can see how your model will be called and passed to an Agent to play with.

In [35]:
import torch
import yaml

# Your model must be imported exactly as follows; from a module called "model" (a file called "model.py") and with
# the class name "Model".

from model import Model

models_checks = {}
# All necessary arguments for your model to initialize with must be saved in a YAML file called "model_config.yaml"
# so that your model can be instantiated exactly as follows. Your model must NOT require any initialization arguments
# besides those described in your "model_config.yaml" file.
model_config = yaml.safe_load(open("model_config.yaml"))

for key in checkpoints_location:
    model = Model(**model_config)
    # Your model checkpoint must be called "checkpoint.pt" and must be a dictionary-like object with your model weights
    # stored at the key "model" so that it can be loaded into your model exactly as follows.
    checkpoint = torch.load(checkpoints_location[key], map_location="cpu")
    model.load_state_dict(checkpoint["model"])
    models_checks[key] = model

# Note: when you load your model weights you may see the following warning. You can safely ignore this warning.

ignore = """
/root/.chess/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
"""

In [36]:
import shutil
import os
from datetime import datetime

def copy_directory(source_dir, destination_dir):
    if not os.path.exists(source_dir):
        print(f"Source directory '{source_dir}' does not exist!")
        return
    
    # Ensure the destination directory exists
    os.makedirs(destination_dir, exist_ok=True)

    # Extract directory name
    dir_name = os.path.basename(source_dir)
    target_path = os.path.join(destination_dir, dir_name)

    # If directory already exists, rename it with timestamp
    if os.path.exists(target_path):
        timestamp = datetime.now().strftime("%H-%M-%S")
        target_path = os.path.join(destination_dir, f"{dir_name}_{timestamp}")

    # Copy the directory
    shutil.copytree(source_dir, target_path)
    print(f"Directory copied successfully to: {target_path}")

# Example Usage
#source = "/shared/yogesh/base_model_run_1/CHK0"  # Change to your source directory
#destination = "/shared/Winning/"  # Change to your destination

#copy_directory(source, destination)


In [37]:
def run_game(m1, m1_key, m2, m2_key):
    agents = {'white': chg.Agent(m1), 'black': chg.Agent(m2)}
    teams = {'white': 'Team White', 'black': 'Team Black'}
    
    print(f'Playing {m1_key} as White and {m2_key} as Black')
    game_result = chg.play_game(
        agents, 
        teams, 
        max_moves=50, 
        min_seconds_per_move=0, 
        verbose=False, 
        poseval=True, 
        image_path="demo.png")
    
    white_points, black_points = game_result['white']['points'], game_result['black']['points']
    white_score, black_score = (white_points + 10_000) / 20_000, (black_points + 10_000) / 20_000
    print(f"{m1_key} as White score: {white_score:,.3}, {m2_key} as Black score: {black_score:,.3}")

    return white_score, black_score

def best_avg(m1, m1_key, m2, m2_key):
    m1_white, m2_black = run_game(m1,m1_key, m2, m2_key)
    m2_white, m1_black = run_game(m2,m2_key, m1, m1_key)

    m1_avg = (m1_white + m1_black)/2
    m2_avg = (m2_white + m2_black)/2

    print(f'{m1_key} with avg of {m1_avg:,.3}')
    print(f'{m2_key} with avg of {m2_avg:,.3}')
    
    if(m1_avg > m2_avg): return 'm1'
    else: return 'm2'


In [ ]:
model_keys = list(models_checks.keys())
game_iterations = 1; # games will be playes 1 as White and 1 as Black (so game_iterations X 2 games will be played)

checkpoint_length = checkpoints_location.__len__

last_winner = 0
next_index = 1

for i in range(len(model_keys)):
    
    if i == len(model_keys): break
    if i < next_index: continue

    m1_key = model_keys[last_winner]
    m2_key = model_keys[next_index]

    print(f'Comparing check points: {m1_key} with {m2_key}')
    winner = best_avg(m1=models_checks[m1_key], m1_key=m1_key, m2=models_checks[m2_key], m2_key=m2_key)

    if winner == 'm2':
        last_winner = next_index

    print(f'Winner is : {model_keys[last_winner]}')

    next_index = next_index + 1
    
winning_checkpoint = model_keys[last_winner]
print(f'last winner is {winning_checkpoint}')

copy_directory(str(path) + "/" + winning_checkpoint, copy_winner)
# The model is passed as the first positional argument to the Agent, and is then available to the agent to use for
# selecting moves in the game.

# agents = {'white': chg.Agent(model), 'black': chg.Agent(model)}
# teams = {'white': 'Team White', 'black': 'Team Black'}

# game_result = chg.play_game(
#     agents, 
#     teams, 
#     max_moves=5, 
#     min_seconds_per_move=0, 
#     verbose=True, 
#     poseval=True, 
#     image_path="demo.png"
# )

Comparing check points: CHK0 with CHK1
Playing CHK0 as White and CHK1 as Black
CHK0 as White score: 0.999, CHK1 as Black score: 0.00065
Playing CHK1 as White and CHK0 as Black
CHK1 as White score: 0.476, CHK0 as Black score: 0.524
CHK0 with avg of 0.762
CHK1 with avg of 0.238
Winner is : CHK0
Comparing check points: CHK0 with CHK2
Playing CHK0 as White and CHK2 as Black
CHK0 as White score: 1.0, CHK2 as Black score: 5e-05
Playing CHK2 as White and CHK0 as Black
CHK2 as White score: 0.532, CHK0 as Black score: 0.468
CHK0 with avg of 0.734
CHK2 with avg of 0.266
Winner is : CHK0
last winner is CHK0
Directory copied successfully to: /shared/Winning/run/CHK0
